In [33]:
from imports_func import *
from dbase import *
import os

In [34]:
X=np.load('/home/miquel/ForMiquel/CLEAN2/X_c1_c2.npy') 
y=np.load('/home/miquel/ForMiquel/CLEAN2/y.npy') #jacobian labels
y_short=np.load('/home/miquel/ForMiquel/CLEAN2/y_short.npy') #subject labels
sub_id=np.load('/home/miquel/ForMiquel/CLEAN2/sub_id.npy') #jacobian idxs
sub_id_short=np.load('/home/miquel/ForMiquel/CLEAN2/sub_id_short.npy') #subject idxs
dt=np.load('/home/miquel/ForMiquel/CLEAN2/dt.npy') #subject idxs
zeros=np.load('/home/miquel/ForMiquel/CLEAN2/zero_cols_JB_c1_c2.npy') #subject idxs
dt_1=np.load('/home/miquel/ForMiquel/CLEAN2/sub_id_1_dt_415.npy')

In [35]:
def KFold_intern(X_train, y_train, folds, linspace, y_short_train, sub_id_train, sub_id_intern):
    
    l=len(linspace)
    l2=len(y_short_train)
    TP=np.zeros((l))
    FP=np.zeros((l))
    FN=np.zeros((l))
        

    kf = cross_validation.KFold(l2, n_folds=folds)

    fold = 1
       
    for train_index_f, test_index_f in kf:  
        aux = sub_id_train[train_index_f]
        idx = np.isin(sub_id_intern, aux)
        X_train_f = X_train[idx,:]
        X_test_f = X_train[~idx,:]
        y_train_f = y_train[idx]
        y_test_f = y_train[~idx]
        
        print("fold:", fold)
        fold = fold + 1
     
        tp_l=[]
        fp_l=[]
        fn_l=[]
        
        for c in linspace:
            
            #clf = SVC(kernel='linear', probability = True, class_weight='balanced', C=c)
            clf = LogisticRegression(solver='liblinear', penalty='l2', C=c, class_weight='balanced')
            clf.fit(X_train_f, y_train_f)
            pred = clf.predict(X_test_f)
            conf = confusion_matrix(y_test_f, pred)
            
            #mask0 = np.where(dt_test_f>1.09)#~400 days
            #mask0 = np.asarray(mask0)
            #pred = pred[mask0][0]
            #y_test_f = y_test_f[mask0][0]
            conf = confusion_matrix(y_test_f, pred)
            tp_l.append(conf[1,1])
            fp_l.append(conf[0,1])
            fn_l.append(conf[1,0])
            
            
        TP = np.vstack((TP, tp_l))
        FP = np.vstack((FP, fp_l))
        FN = np.vstack((FN, fn_l))
        
            
    TP=TP[1:,:]
    FP=FP[1:,:]
    FN=FN[1:,:]
    
    TP=np.sum(TP, axis=0)
    FP=np.sum(FP, axis=0)
    FN=np.sum(FN, axis=0)
    
    F=(2*TP)/(2*TP + FP + FN)
    maxi = np.amax(F)
    maxi_idx = np.argmax(F)
    
    #plt.figure(1)
    #plt.plot(linspace, F, color='blue', label='F-measure')
    #plt.legend(loc='best')
    #plt.xlabel('C')
    #plt.title('metrics logistic regression combination c1 and c2')
    #plt.show()
    
    return maxi, maxi_idx
    


In [31]:
def nested_CV_F_measure(X, y, dt_1, splits, folds, linspace, dt, percentage, y_short, sub_id, sub_id_short, zeros):
   
    box = np.zeros((121,145,121))

    SPECIFICITY=[]
    PRECISION=[]
    RECALL=[]
    AUC=[]
    F=[]
    ACCURACY=[]
    
    SPECIFICITY1=[]
    PRECISION1=[]
    RECALL1=[]
    AUC1=[]
    F1=[]
    ACCURACY1=[]
    
    tp_l=[]
    fp_l=[]
    fn_l=[]
    
    
    spl = StratifiedShuffleSplit(n_splits=splits, test_size=0.2, random_state=42)
    split=1
    
    for train_index, test_index in spl.split(sub_id_short, y_short): #split in subjects
        sub_id_train = sub_id_short[train_index] #search subject train idxs
        idx = np.isin(sub_id, sub_id_train) #match train idxs 
        X_test = X[~idx,:]
        y_test = y[~idx]
        dt_test = dt[~idx]
        
        mask = np.where(dt>415)#~415 days
        mask = np.asarray(mask)
        print(mask)
                
        mask_bool = np.zeros(288, dtype=bool)
        mask_bool[mask]=True
        print(mask_bool)
        
        idx = np.logical_and(idx, mask_bool)

        dt_train = dt[idx]

        X_train = X[idx,:]
        y_train = y[idx]
        sub_id_intern=sub_id[idx] 
        
        train_index_bool = np.zeros(124, dtype=bool)
        
        train_index_bool[train_index]=True
        
        print("train index:", train_index_bool)
        train_index = np.logical_and(train_index_bool, dt_1)
        print("train index:", train_index)


        y_short_train=y_short[train_index]
        
        
        print("split:", split)
        split = split + 1
        

        print("jacobians at train after removing dt<415 days:", X_train.shape[0])
        
        S = SelectPercentile(percentile=percentage)
        S.fit(X_train, y_train)
        
        X_train_s = X_train[:,S.get_support()]
        X_test_s = X_test[:,S.get_support()]
     
        maxi, maxi_idx = KFold_intern(X_train_s, y_train, folds, linspace, y_short_train, sub_id_train, sub_id_intern)
        c = linspace[maxi_idx]
        print("C:", c)
        
        #clf = SVC(kernel='linear', probability = True, class_weight='balanced', C=c)
        clf = LogisticRegression(solver='liblinear', penalty='l2', C=c, class_weight='balanced')
        clf.fit(X_train_s, y_train)
        pred = clf.predict(X_test_s)
        probas_ = clf.predict_proba(X_test_s)
        

        probas_ = probas_[:, 1]
        
        #preparation masks to test AUC as function of dt
        mask1 = np.where(dt_test>415)#~400 days
        mask1 = np.asarray(mask1)
        pred_1 = pred[mask1][0]
        y_test_1 = y_test[mask1][0]
        probas_1 = probas_[mask1][0]
        
        conf = confusion_matrix(y_test, pred)
        tp_l.append(conf[1,1])
        fp_l.append(conf[0,1])
        fn_l.append(conf[1,0])
        
        f1=f1_score(y_test, pred)
        F.append(f1)
        
        f1_1=f1_score(y_test_1, pred_1)
        if np.isnan(f1_1) == False:
            F1.append(f1_1)
        
        
        fpr, tpr, thresholds = roc_curve(y_test, probas_)
        roc_auc = sklearn.metrics.auc(fpr, tpr)
        AUC.append(roc_auc)
        
        fpr, tpr, thresholds = roc_curve(y_test_1, probas_1)
        roc_auc_1 = sklearn.metrics.auc(fpr, tpr)
        if np.isnan(roc_auc_1) == False:
            AUC1.append(roc_auc_1)

        precision=precision_score(y_test, pred)
        recall=recall_score(y_test, pred)
        PRECISION.append(precision)
        RECALL.append(recall)
        specificity= (conf[0,0])/(conf[0,0]+conf[0,1])
        SPECIFICITY.append(specificity)
        acc=accuracy_score(y_test, pred)
        ACCURACY.append(acc)
        
        precision1=precision_score(y_test_1, pred_1)
        recall1=recall_score(y_test_1, pred_1)
        PRECISION1.append(precision1)
        RECALL1.append(recall1)
        conf1 = confusion_matrix(y_test_1, pred_1)
        specificity1= (conf1[0,0])/(conf1[0,0]+conf1[0,1])
        SPECIFICITY1.append(specificity1)
        acc1=accuracy_score(y_test_1, pred_1)
        ACCURACY1.append(acc1)
        
    TP=np.sum(tp_l, axis=0)
    FP=np.sum(fp_l, axis=0)
    FN=np.sum(fn_l, axis=0)

    
    F_g=(2*TP)/(2*TP + FP + FN)
    

    print("ACCURACY_mean:", np.mean(ACCURACY))
    print("PRECISION_mean:", np.mean(PRECISION))
    print("RECALL_mean:", np.mean(RECALL))
    print("SPECIFICITY_mean:", np.mean(SPECIFICITY))
    print("F_mean:", np.mean(F))
    print("AUC_mean:", np.mean(AUC))
    
    print("ACCURACY1_mean:", np.mean(ACCURACY1))
    print("PRECISION1_mean:", np.mean(PRECISION1))
    print("RECALL1_mean:", np.mean(RECALL1))
    print("SPECIFICITY1_mean:", np.mean(SPECIFICITY1))
    print("F1_mean:", np.mean(F1))
    print("AUC1_mean:", np.mean(AUC1))

    print("ACCURACY_std:", np.std(ACCURACY))
    print("PRECISION_std:", np.std(PRECISION))
    print("RECALL_std:", np.std(RECALL))
    print("SPECIFICITY_std:", np.std(SPECIFICITY))
    print("F_std:", np.std(F))
    print("AUC_std:", np.std(AUC))

    print("ACCURACY1_std:", np.std(ACCURACY1))
    print("PRECISION1_std:", np.std(PRECISION1))
    print("RECALL1_std:", np.std(RECALL1))
    print("SPECIFICITY1_std:", np.std(SPECIFICITY1))
    print("F1_std:", np.std(F1))
    print("AUC1_std:", np.std(AUC1))

    return np.mean(F), np.mean(AUC), np.mean(PRECISION), np.mean(RECALL), np.mean(SPECIFICITY), np.mean(ACCURACY), np.std(F), np.std(AUC), np.std(PRECISION), np.std(RECALL), np.std(SPECIFICITY), np.std(ACCURACY),np.mean(F1), np.mean(AUC1), np.mean(PRECISION1), np.mean(RECALL1), np.mean(SPECIFICITY1), np.mean(ACCURACY1), np.std(F1), np.std(AUC1), np.std(PRECISION1), np.std(RECALL1), np.std(SPECIFICITY1), np.std(ACCURACY1)



In [32]:
splits = 100
folds= 3
linspace = np.linspace(10e-2, 10e3, num=20)
percentage = 0.35
F, AUC, PRECISION, RECALL, SPECIFICITY, ACCURACY, Fs, AUCs, PRECISIONs, RECALLs, SPECIFICITYs, ACCURACYs, F1, AUC1, PRECISION1, RECALL1, SPECIFICITY1, ACCURACY1, F1s, AUC1s, PRECISION1s, RECALL1s, SPECIFICITY1s, ACCURACY1s = nested_CV_F_measure(X, y, dt_1, splits, folds, linspace, dt, percentage, y_short, sub_id, sub_id_short, zeros)
os.system('espeak "la ejecución ha terminadoooooooooooooooooooooooooo"')

[[  7  11  14  15  17  18  23  24  25  26  29  30  31  32  33  34  36  40
   41  42  43  44  45  47  53  57  58  60  61  62  63  64  66  67  68  69
   72  73  74  75  78  79  80  81  82  83  85  89  92  93  96  97  98  99
  105 108 109 110 111 112 113 115 116 119 120 122 123 125 126 132 133 134
  135 138 139 141 146 147 149 150 152 153 155 159 160 161 162 163 165 166
  167 168 170 171 172 174 175 177 178 180 181 183 184 185 186 187 188 190
  191 193 195 197 198 202 204 205 206 207 212 213 214 215 219 220 221 223
  224 226 229 232 233 234 235 237 238 239 240 241 242 243 244 245 246 247
  248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
  266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283
  284 285 286 287]]
[False False False False False False False  True False False False  True
 False False  True  True False  True  True False False False False  True
  True  True  True False False  True  True  True  True  True  True False
  True False False Fa

/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    13     90     99 ..., 580922 580925 580926] are constant.
  UserWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


fold: 1
fold: 2
fold: 3
C: 0.1
[[  7  11  14  15  17  18  23  24  25  26  29  30  31  32  33  34  36  40
   41  42  43  44  45  47  53  57  58  60  61  62  63  64  66  67  68  69
   72  73  74  75  78  79  80  81  82  83  85  89  92  93  96  97  98  99
  105 108 109 110 111 112 113 115 116 119 120 122 123 125 126 132 133 134
  135 138 139 141 146 147 149 150 152 153 155 159 160 161 162 163 165 166
  167 168 170 171 172 174 175 177 178 180 181 183 184 185 186 187 188 190
  191 193 195 197 198 202 204 205 206 207 212 213 214 215 219 220 221 223
  224 226 229 232 233 234 235 237 238 239 240 241 242 243 244 245 246 247
  248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
  266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283
  284 285 286 287]]
[False False False False False False False  True False False False  True
 False False  True  True False  True  True False False False False  True
  True  True  True False False  True  True  True  True  True  T

/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [   129    140    175 ..., 580917 580918 580919] are constant.
  UserWarning)


fold: 1
fold: 2
fold: 3
C: 0.1
[[  7  11  14  15  17  18  23  24  25  26  29  30  31  32  33  34  36  40
   41  42  43  44  45  47  53  57  58  60  61  62  63  64  66  67  68  69
   72  73  74  75  78  79  80  81  82  83  85  89  92  93  96  97  98  99
  105 108 109 110 111 112 113 115 116 119 120 122 123 125 126 132 133 134
  135 138 139 141 146 147 149 150 152 153 155 159 160 161 162 163 165 166
  167 168 170 171 172 174 175 177 178 180 181 183 184 185 186 187 188 190
  191 193 195 197 198 202 204 205 206 207 212 213 214 215 219 220 221 223
  224 226 229 232 233 234 235 237 238 239 240 241 242 243 244 245 246 247
  248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
  266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283
  284 285 286 287]]
[False False False False False False False  True False False False  True
 False False  True  True False  True  True False False False False  True
  True  True  True False False  True  True  True  True  True  T

...

fold: 1
fold: 2
fold: 3
C: 0.1
[[  7  11  14  15  17  18  23  24  25  26  29  30  31  32  33  34  36  40
   41  42  43  44  45  47  53  57  58  60  61  62  63  64  66  67  68  69
   72  73  74  75  78  79  80  81  82  83  85  89  92  93  96  97  98  99
  105 108 109 110 111 112 113 115 116 119 120 122 123 125 126 132 133 134
  135 138 139 141 146 147 149 150 152 153 155 159 160 161 162 163 165 166
  167 168 170 171 172 174 175 177 178 180 181 183 184 185 186 187 188 190
  191 193 195 197 198 202 204 205 206 207 212 213 214 215 219 220 221 223
  224 226 229 232 233 234 235 237 238 239 240 241 242 243 244 245 246 247
  248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
  266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283
  284 285 286 287]]
[False False False False False False False  True False False False  True
 False False  True  True False  True  True False False False False  True
  True  True  True False False  True  True  True  True  True  T

/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    85    129    140 ..., 580906 580907 580924] are constant.
  UserWarning)


fold: 1
fold: 2
fold: 3
C: 625.09375
[[  7  11  14  15  17  18  23  24  25  26  29  30  31  32  33  34  36  40
   41  42  43  44  45  47  53  57  58  60  61  62  63  64  66  67  68  69
   72  73  74  75  78  79  80  81  82  83  85  89  92  93  96  97  98  99
  105 108 109 110 111 112 113 115 116 119 120 122 123 125 126 132 133 134
  135 138 139 141 146 147 149 150 152 153 155 159 160 161 162 163 165 166
  167 168 170 171 172 174 175 177 178 180 181 183 184 185 186 187 188 190
  191 193 195 197 198 202 204 205 206 207 212 213 214 215 219 220 221 223
  224 226 229 232 233 234 235 237 238 239 240 241 242 243 244 245 246 247
  248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
  266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283
  284 285 286 287]]
[False False False False False False False  True False False False  True
 False False  True  True False  True  True False False False False  True
  True  True  True False False  True  True  True  True  T

fold: 1
fold: 2
fold: 3
C: 1250.0875
[[  7  11  14  15  17  18  23  24  25  26  29  30  31  32  33  34  36  40
   41  42  43  44  45  47  53  57  58  60  61  62  63  64  66  67  68  69
   72  73  74  75  78  79  80  81  82  83  85  89  92  93  96  97  98  99
  105 108 109 110 111 112 113 115 116 119 120 122 123 125 126 132 133 134
  135 138 139 141 146 147 149 150 152 153 155 159 160 161 162 163 165 166
  167 168 170 171 172 174 175 177 178 180 181 183 184 185 186 187 188 190
  191 193 195 197 198 202 204 205 206 207 212 213 214 215 219 220 221 223
  224 226 229 232 233 234 235 237 238 239 240 241 242 243 244 245 246 247
  248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
  266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283
  284 285 286 287]]
[False False False False False False False  True False False False  True
 False False  True  True False  True  True False False False False  True
  True  True  True False False  True  True  True  True  T

0